## Two Body Loss

In [1]:
using KeldyshContraction
using KeldyshContraction: Classical, Quantum, Plus, Minus

@qfields ϕ::Destroy(Classical) ψ::Destroy(Quantum)
@syms Γ::Real

loss2boson = im*(
    0.5 * ϕ' * ψ' * (ϕ(Minus) * ϕ(Minus) + ψ(Minus) * ψ(Minus)) -
    0.5 * ϕ(Plus) * ψ(Plus) * (ϕ' * ϕ' + ψ' * ψ') +
    ϕ' * ψ' * (ϕ(Plus) * ψ(Plus) + ϕ(Minus) * ψ(Minus))
)
L_int = InteractionLagrangian(loss2boson)

GF = DressedPropagator(L_int)

Σ = SelfEnergy(GF)

GF = DressedPropagator(L_int; order = 2)

Σ = SelfEnergy(GF; order = 2)

Self Energy:
keldysh:  -0.5*Gᴬ(y₁,y₂)*Gᴬ(y₁,y₂)*Gᴷ(y₂,y₁) + -1.0*Gᴿ(y₁,y₂)*Gᴬ(y₂,y₁)*Gᴿ(y₂,y₂) + Gᴿ(y₁,y₂)*Gᴬ(y₂,y₁)*Gᴷ(y₂,y₂) + Gᴿ(y₁,y₂)*Gᴷ(y₂,y₁)*Gᴷ(y₂,y₂) + -0.5*Gᴷ(y₁,y₂)*Gᴷ(y₁,y₂)*Gᴷ(y₂,y₁) + -1.0*Gᴿ(y₁,y₂)*Gᴷ(y₂,y₁)*Gᴿ(y₂,y₂) + -1.0*Gᴷ(y₁,y₂)*Gᴬ(y₁,y₂)*Gᴷ(y₂,y₁)
retarded: 
advanced: -0.5*Gᴬ(y₁,y₂)*Gᴬ(y₁,y₂)*Gᴿ(y₂,y₁) + -1.0*Gᴷ(y₁,y₂)*Gᴬ(y₁,y₂)*Gᴿ(y₂,y₁) + Gᴿ(y₁,y₂)*Gᴬ(y₂,y₁)*Gᴿ(y₂,y₂) + -1.0*Gᴿ(y₁,y₂)*Gᴬ(y₂,y₁)*Gᴷ(y₂,y₂) + -1.0*Gᴿ(y₁,y₂)*Gᴷ(y₂,y₁)*Gᴷ(y₂,y₂) + Gᴿ(y₁,y₂)*Gᴷ(y₂,y₁)*Gᴿ(y₂,y₂) + -0.5*Gᴷ(y₁,y₂)*Gᴷ(y₁,y₂)*Gᴿ(y₂,y₁)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*